<a href="https://colab.research.google.com/github/Bariharsh/Automated-Pneumonia-Detection/blob/main/deepvision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from datasets import load_dataset

ds = load_dataset("hf-vision/chest-xray-pneumonia")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00007.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

data/train-00001-of-00007.parquet:   0%|          | 0.00/385M [00:00<?, ?B/s]

data/train-00002-of-00007.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

...

data/validation-00000-of-00001.parquet:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/78.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5216 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/624 [00:00<?, ? examples/s]

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
# Merge original train and create our own split
full_ds = ds["train"].train_test_split(test_size=0.2, seed=42)

train_hf = full_ds["train"]
temp_hf  = full_ds["test"]

val_test = temp_hf.train_test_split(test_size=0.5, seed=42)

val_hf  = val_test["train"]
test_hf = val_test["test"]


In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])


In [6]:

IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(example,augment=False):
    '''
    Preprocess the PIL image (python image library) into array numbers
    '''
    image = example["image"].convert("L")  # grayscale
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image = np.array(image, dtype=np.float32) / 255.0
    image = np.expand_dims(image, axis=-1)  # (224,224,1)
    if augment:
        image = data_augmentation(image)
    label = example["label"]
    return image, label

In [7]:
def hf_to_tf(dataset,augment=False):
    '''
    it converts the hugging face datasets to tensor flow training pipeline
    '''
    return tf.data.Dataset.from_generator( # creates an tensorflow datasets by pulling dataa one item at a time in pythin generator
        lambda: (preprocess(x, augment) for x in dataset),
        output_signature=(
            tf.TensorSpec(shape=(224,224,1), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        ) # output signature just gives the shapes and type of data that matches the resnet18 architecture
    )


train_ds = hf_to_tf(train_hf, augment=True) \
    .shuffle(1000) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = hf_to_tf(val_hf) \
    .batch(BATCH_SIZE)

test_ds = hf_to_tf(test_hf) \
    .batch(BATCH_SIZE)

# prefech:- it prepares the next batch while current batch being trained
# shuffle:- it mixes the training images randomly before feeding them to the model.

#test


In [27]:
def residual_block(x, filters, stride=1):
    shortcut = x

    x = layers.Conv2D(filters, 3, strides=stride, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x


In [28]:
def ResNet18(input_shape=(224,224,1), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding="same", use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # Conv2_x
    x = residual_block(x, 64)
    x = residual_block(x, 64)

    # Conv3_x
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)

    # Conv4_x
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)

    # Conv5_x
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)


In [29]:
model = ResNet18(num_classes=2)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_60 (Conv2D)  │ (None, 112, 112,  │      3,136 │ input_layer_3[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_60[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_51 (ReLU)     │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 56, 56,    │          0 │ re_lu_51[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_61 (Conv2D)  │ (None, 56, 56,    │     36,864 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_61[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_52 (ReLU)     │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_62 (Conv2D)  │ (None, 56, 56,    │     36,864 │ re_lu_52[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_62[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_53 (ReLU)     │ (None, 56, 56,    │          0 │ add_24[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_63 (Conv2D)  │ (None, 56, 56,    │     36,864 │ re_lu_53[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_63[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_54 (ReLU)     │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_64 (Conv2D)  │ (None, 56, 56,    │     36,864 │ re_lu_54[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_64[0][0] 

 Total params: 11,180,866 (42.65 MB)

 Trainable params: 11,171,266 (42.61 MB)

 Non-trainable params: 9,600 (37.50 KB)

In [30]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
)


Epoch 1/25
    131/Unknown 187s 1s/step - accuracy: 0.8024 - loss: 0.8484   

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


131/131 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.8027 - loss: 0.8457 - val_accuracy: 0.3276 - val_loss: 17.7328
Epoch 2/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.8755 - loss: 0.3185 - val_accuracy: 0.7778 - val_loss: 0.5198
Epoch 3/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 162s 980ms/step - accuracy: 0.9079 - loss: 0.2239 - val_accuracy: 0.2701 - val_loss: 4.7163
Epoch 4/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 163s 994ms/step - accuracy: 0.9315 - loss: 0.1645 - val_accuracy: 0.8697 - val_loss: 0.4325
Epoch 5/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 167s 1s/step - accuracy: 0.9448 - loss: 0.1410 - val_accuracy: 0.7797 - val_loss: 0.9263
Epoch 6/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - accuracy: 0.9480 - loss: 0.1319 - val_accuracy: 0.9349 - val_loss: 0.1538
Epoch 7/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 164s 987ms/step - accuracy: 0.9463 - loss: 0.1348 - val_accuracy: 0.9464 - val_loss: 0.1591
Epoch 8/25
131/131 ━━━━━━━━━━━━━━━━━━━━ 162s 981ms/step - accuracy: 0.9520 - loss: 0.1208 - val_ac

KeyboardInterrupt: 

In [16]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)



20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step - accuracy: 0.3297 - loss: 3.0506
Test Accuracy: 0.629807710647583
